# ETL Pipeline
Applied commands that constitude an ETL Pipeline.
### Libraries & Datasets.
- Import Python libraries
- Load `messages.csv` into a dataframe for inspection.
- Load `categories.csv` into a dataframe for inspection.

In [1]:
# import libraries
import numpy as np
import pandas as pd
import sqlite3
import sqlalchemy
import matplotlib.pyplot as plt
from sqlalchemy import create_engine

In [4]:
# Load datasets (messages)
messages = pd.read_csv("./Data/messages.csv")
messages.head()

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [5]:
# Load datasets (categories)
categories = pd.read_csv("./Data/categories.csv")
categories.head()

,id,categories
0,2,related-1;request-0;offer-0;aid_related-0;medi...
1,7,related-1;request-0;offer-0;aid_related-1;medi...
2,8,related-1;request-0;offer-0;aid_related-0;medi...
3,9,related-1;request-1;offer-0;aid_related-1;medi...
4,12,related-1;request-0;offer-0;aid_related-0;medi...


### Merge Datasets
- Both datasets will be merged by their common IDs.
- The results of the merge datasets will then form another dataframe.

In [6]:
# merging datasets
df = messages.merge(categories, how ='outer', on =['id'])
df.head()

,id,message,original,genre,categories
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,related-1;request-0;offer-0;aid_related-0;medi...
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,related-1;request-0;offer-0;aid_related-1;medi...
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,related-1;request-0;offer-0;aid_related-0;medi...
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,related-1;request-1;offer-0;aid_related-1;medi...
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,related-1;request-0;offer-0;aid_related-0;medi...


### split `categories` into separate category columns.
- Split the values in the `categories` column on the `;` character so that each value becomes a separate column.
- Use the first row of categories dataframe to create column names for the categories data.
- Rename columns of `categories` with new column names.